# [619. Biggest Single Number](https://leetcode.com/problems/biggest-single-number/?envType=study-plan-v2&envId=top-sql-50)

Table: MyNumbers

<pre>+-------------+------+
| Column Name | Type |
+-------------+------+
| num         | int  |
+-------------+------+</pre>
This table may contain duplicates (In other words, there is no primary key for this table in SQL).
Each row of this table contains an integer.
 

A single number is a number that appeared only once in the MyNumbers table.

Find the largest single number. If there is no single number, report null.

The result format is in the following example.

 

Example 1:

Input: 
MyNumbers table:
<pre>+-----+
| num |
+-----+
| 8   |
| 8   |
| 3   |
| 3   |
| 1   |
| 4   |
| 5   |
| 6   |
+-----+</pre>
Output: 
<pre>+-----+
| num |
+-----+
| 6   |
+-----+</pre>
Explanation: The single numbers are 1, 4, 5, and 6.
Since 6 is the largest single number, we return it.
Example 2:

Input: 
MyNumbers table:
<pre>+-----+
| num |
+-----+
| 8   |
| 8   |
| 7   |
| 7   |
| 3   |
| 3   |
| 3   |
+-----+</pre>
Output: 
<pre>+------+
| num  |
+------+
| null |
+------+</pre>
Explanation: There are no single numbers in the input table so we return null.

In [1]:
# pandas schema

import pandas as pd

data = [[8], [8], [3], [3], [1], [4], [5], [6]]  # first example
#data = [[8], [8], [7], [7], [3], [3], [3]] # second example
my_numbers = pd.DataFrame(data, columns=['num']).astype({'num': 'Int64'})

In [2]:
#converting to spark dataframe
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

my_numbers_df = spark.createDataFrame(my_numbers)
my_numbers_df.show()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/11 17:57:00 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/11 17:57:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 17:57:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+
|num|
+---+
|  8|
|  8|
|  3|
|  3|
|  1|
|  4|
|  5|
|  6|
+---+


In [3]:
# solving in spark dataframe

my_numbers_df \
    .groupBy('num') \
    .agg(F.count('num').alias('count')) \
    .filter(F.col('count') == 1) \
    .agg(F.max('num').alias('num')) \
    .show()

+---+
|num|
+---+
|  6|
+---+


In [4]:
# solving in spark SQL

my_numbers_df.createOrReplaceTempView('MyNumbers')

spark.sql('''
select max(t.num) as num
from (
    select num
    from mynumbers
    group by num
    having count(num)=1
) t;
''').show()

+---+
|num|
+---+
|  6|
+---+


In [5]:
spark.stop()